In [49]:
import pandas as pd
import numpy as np
import openpyxl
import re

In [50]:
df = pd.read_excel("credata.xlsx", engine = "openpyxl")

# Drop Useless Columns

In [51]:
df.drop(df.columns[-8:], axis = 1, inplace = True)
df.drop(columns = ["Owner Name","Mailing Address","Email","EMAIL","phone, 2","secondary broker"], inplace = True)
df.drop(columns = ["Number of Units"], inplace = True)
df.drop(columns = ["Total area"], inplace = True)

## Clean Price Column And Convert to Integer

In [52]:
df = df[df["Price"].str.contains("-")==False]
df["Price"] = df["Price"].apply(lambda x: x[1:])
df["Price"] = df["Price"].apply(lambda x: x.replace(",", ""))
df["Price"] = ["DROP" if "." in x else x for x in df["Price"]]
df.drop(df[df["Price"] == "DROP"].index, inplace = True)
df.drop(df[df["Price"] == "ot Type"].index, inplace = True)
df["Price"] = ["DROP" if " " in x else x for x in df["Price"]]
df.drop(df[df["Price"] == ""].index, inplace = True)
df.drop(df[df["Price"] == "DROP"].index, inplace = True)

In [53]:
df["Price"] = df["Price"].apply(lambda x : int(x))

## Create City, State, Zip Columns

In [54]:
df["split"] = [x.split(",") for x in df["Address"]]
df["city"] = [x[0] for x in df["split"]]

df.drop(df[df["split"].apply(len) == 1].index, inplace = True)

df["state"] = [x[1].strip().split()[0] for x in df["split"]]
df["zipcode"] = [x[1].strip().split()[1] for x in df["split"]]

## Drop Number of Stories Column

In [55]:
df.drop(columns = ["Number Of Stories"], inplace = True)

## Clean Lot Size

In [56]:
df.drop(df[(df["Lot Size"] == "") | (df["Lot Size"] == " ")].index, inplace = True)
df = df[df["Lot Size"].str.contains("-")==False]
df["Lot Size"] = df["Lot Size"].apply(lambda x: x.strip())
df["Lot Size"] = df["Lot Size"].apply(lambda x: x.split())
df["Lot Size"] = [x if len(x) == 2 else "DROP" for x in df["Lot Size"]]
df.drop(df[df["Lot Size"] == "DROP"].index, inplace = True)
df["Lot Size"] = df["Lot Size"].apply(lambda x: " ".join(x))
df["Lot Size"] = df["Lot Size"].apply(lambda x: x.replace(",", ""))

## Convert Acres to Square Foot for Consistency

In [57]:
df["Lot Size"] = [float(x.split()[0]) if x.split()[1] == "SF" 
                  else (float(x.split()[0]) * 43560)
                  for x in df["Lot Size"]]

## Save Cleaned Data

In [30]:
df.drop(columns = ["split","phone","broker"], inplace = True)
df.drop(columns = ["Year Built"], inplace = True)
df.drop(columns = ["Address"], inplace = True)
df.rename(columns = {"City":"Address", "Lot Size":"Lot Size (sqft)", "url":"Url","city":"City","state":"State","zipcode":"Zipcode"}, 
          inplace = True)
df.to_excel("credata_cleaned.xlsx")